##Importamos las librerias necesarias.

In [ ]:
import pandas as pd
import numpy as np
!pip install boto3
import boto3
import psycopg2
import configparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 57.1 MB/s eta 0:00:00


##Archivo de configuración

In [ ]:
config = configparser.ConfigParser()
config.read('escec_dataWarehouse.cfg')

['escec_dataWarehouse.cfg']

##RDS de la base de datos myMovieStore

In [ ]:
RDS_HOST = 'mymoviestore.c6j34omip28z.us-east-2.rds.amazonaws.com'
#RDS_HOST=config.get('DW', 'RDS_HOST')
#RDS_HOST
print(RDS_HOST)

mymoviestore.c6j34omip28z.us-east-2.rds.amazonaws.com


In [ ]:
postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""

#Creando Dimensiones

##Dimensión de Custumers

La creación de esta dimensión es bastante sencilla porque no tenemos necesidad de integrar otras tablas.

In [ ]:
sql_query = 'SELECT * FROM custumer;'
dim_custumer = pd.read_sql(sql_query, postgres_driver)
dim_custumer=dim_custumer.rename(columns={'Custumer type': 'custumer_type'})
dim_custumer.head()

,id_client,name,gender,custumer_type
0,1,Lora,Female,Member
1,2,Ambros,Male,Member
2,3,Mervin,Male,Member
3,4,Arliene,Female,Member
4,5,Aldous,Male,Member


##Dimensión Payment_Methods

La creación de esta dimensión también es directa desde la tabla.

In [ ]:
sql_query = 'SELECT * FROM payment_methods;'
dim_payment_methods = pd.read_sql(sql_query, postgres_driver)
dim_payment_methods.head()

,id_payment,name
0,1,Ewallet
1,2,Cash
2,3,Credit card


##Dimensión Movies

Esta dimensión es la más complicada de armar, porque existen otras tablas: categories,countries, y movie_type que necesitan ser integradas a la tabla movies.

In [ ]:
sql_query = 'SELECT * FROM movies;'
df_movies = pd.read_sql(sql_query, postgres_driver)
df_movies.head(3)

,show_id,type,title,director,casting,country,date_added,release_year,rating,duration,listed_in,description,price
0,1891,1,Qurious Como,Varios,Varios,0,2020-10-03,2017,TV-Y,1 Season,17,"Shrunk down in size, Como and his friends exam...",25
1,1892,2,A Chaster Marriage,Umut Kirca,"Emre Karayel, Ceren Moray, Begüm Kütük, Ececan...",76,2020-10-02,2016,TV-14,94 min,8,"Forced to wed his childhood friend, a man obse...",5
2,1893,2,A Go! Go! Cory Carson Halloween,Alex Woo,"Alan C. Lim, Smith Foreman, Maisie Benson, Ann...",0,2020-10-02,2020,TV-Y,22 min,5,"Cory, Chrissy and Freddie are on the hunt for ...",5


Incorporamos la tabla de los countries:

In [ ]:
sql_query = 'SELECT * FROM countries;'
df_countries = pd.read_sql(sql_query, postgres_driver)
df_countries = df_countries.rename(columns={'id_country': 'country', 'name': 'country_name'})
df_countries.head(3)

,country,country_name
0,0,Desconocido
1,1,Argentina
2,2,Australia


Uno los dataframes de movies con countries:

In [ ]:
df_movies_countries=df_movies.merge(df_countries, on='country', how='inner')
df_movies_countries =df_movies_countries.drop(['country'], axis=1)
df_movies_countries.tail(3)

,show_id,type,title,director,casting,date_added,release_year,rating,duration,listed_in,description,price,country_name
8804,7984,2,"Septiembre, un llanto en silencio",Kenneth Müller,"Saúl Lisazo, Mario Zaragoza, Juan Pablo Olysla...",2018-03-10,2017,TV-MA,69 min,12,"After a terrorist attack in Guatemala, a baby ...",5,Guatemala
8805,8074,2,Sprinter,Storm Saulter,"Dale Elliott, Kadeem Wilson, Dennis Titus, Dav...",2020-04-15,2018,TV-MA,112 min,12,When a Jamaican teen hopes to use track to reu...,5,Jamaica
8806,8457,2,The Pirates of Somalia,Bryan Buckley,"Evan Peters, Barkhad Abdi, Melanie Griffith, A...",2018-03-01,2017,R,118 min,12,An aspiring journalist travels to Somalia in h...,5,Somalia


Ahora es necesario combinarlo con la tabla de movie type.

In [ ]:
sql_query = 'SELECT * FROM movie_type;'
df_movie_type = pd.read_sql(sql_query, postgres_driver)
df_movie_type.head(3)

,id_type,name
0,1,TV Show
1,2,Movie


Le hacemos merge con la tabla que ya habiamos combinado: df_movies_countries

In [ ]:
df_movies = df_movies_countries.merge(df_movie_type, left_on='type',right_on='id_type', how='inner')
dim_movies = df_movies
dim_movies=dim_movies.drop(['type','id_type'], axis=1)
dim_movies = dim_movies.rename(columns={'show_id': 'id_movies','name': 'show_type'})
dim_movies.tail(3)

,id_movies,title,director,casting,date_added,release_year,rating,duration,listed_in,description,price,country_name,show_type
8804,7984,"Septiembre, un llanto en silencio",Kenneth Müller,"Saúl Lisazo, Mario Zaragoza, Juan Pablo Olysla...",2018-03-10,2017,TV-MA,69 min,12,"After a terrorist attack in Guatemala, a baby ...",5,Guatemala,Movie
8805,8074,Sprinter,Storm Saulter,"Dale Elliott, Kadeem Wilson, Dennis Titus, Dav...",2020-04-15,2018,TV-MA,112 min,12,When a Jamaican teen hopes to use track to reu...,5,Jamaica,Movie
8806,8457,The Pirates of Somalia,Bryan Buckley,"Evan Peters, Barkhad Abdi, Melanie Griffith, A...",2018-03-01,2017,R,118 min,12,An aspiring journalist travels to Somalia in h...,5,Somalia,Movie


##Dimensión Ventas/Casi Tabla de Hechos

Nuestra tabla de ventas con la que contamos en la base de datos, solo contiene las ventas de una sucursal, especificamente del mes de marzo. Necesitamos juntar estos datos con las ventas de Enero y Febrero. Estos archivos se encuentran en S3

In [ ]:
s3 = boto3.resource(
    service_name = 's3',
    region_name = 'us-east-2',
    aws_access_key_id = config.get('IAM', 'ACCESS_KEY'),
    aws_secret_access_key = config.get('IAM', 'SECRET_ACCESS_KEY')
)

In [ ]:
for bucket in s3.buckets.all():
    S3_BUCKET_NAME = bucket.name
    print(bucket.name)

mymoviestore


In [ ]:
S3_BUCKET_NAME = 'mymoviestore'

In [ ]:
#extraemos todo lo que está en el bucket
remoteFileList = []
for objt in s3.Bucket(S3_BUCKET_NAME).objects.all():
    remoteFileList.append(objt.key)

remoteFileList

['ventas/',
 'ventas/Transacciones_ventas_enero.csv',
 'ventas/Transacciones_ventas_febrero.csv']

Obtenemos la data de las **ventas de Enero** y la guardamos en un DataFrame.

In [ ]:
import io
file = s3.Bucket(S3_BUCKET_NAME).Object(remoteFileList[1]).get()
data = file['Body'].read()
df_ventas_enero = pd.read_csv(io.BytesIO(data))
df_ventas_enero.head(3)

,id_ventas,movies_show_id,custumer_id_client,payment_methods_id_payment,total,fecha
0,1,357,148,1,15,1/1/2023
1,2,7082,265,3,40,1/1/2023
2,3,2512,187,2,5,1/1/2023


Leemos las **ventas de Febrero**

In [ ]:
file = s3.Bucket(S3_BUCKET_NAME).Object(remoteFileList[2]).get()
data = file['Body'].read()
df_ventas_febrero= pd.read_csv(io.BytesIO(data))
df_ventas_febrero.head(3)

,id_ventas,movies_show_id,custumer_id_client,payment_methods_id_payment,total,fecha
0,1,252,178,2,35,2/1/2023
1,2,3170,216,1,40,2/1/2023
2,3,3838,238,2,35,2/1/2023


Incluimos las **ventas de Marzo**, que se encuentran en nuestra base de datos.

In [ ]:
sql_query = 'SELECT * FROM ventas;'
df_ventas_marzo = pd.read_sql(sql_query, postgres_driver)
df_ventas_marzo.head(3)

,id_ventas,movies_show_id,custumer_id_client,payment_methods_id_payment,total,fecha
0,1,2453,70,3,35,2023-03-01
1,2,4642,241,3,15,2023-03-01
2,3,5185,144,3,30,2023-03-01


Unimos todas las ventas de la tienda del año en un mismo data_frame Ventas

In [ ]:
df_ventas = pd.concat([df_ventas_enero,df_ventas_febrero,df_ventas_marzo])
df_ventas = df_ventas.rename(columns={'movies_show_id': 'id_movies', 'custumer_id_client': 'id_client','payment_methods_id_payment': 'id_payment'})

In [ ]:
df_ventas.head(3)

,id_ventas,id_movies,id_client,id_payment,total,fecha
0,1,357,148,1,15,1/1/2023
1,2,7082,265,3,40,1/1/2023
2,3,2512,187,2,5,1/1/2023


##Fact Table

La Fact Table es casi la misma que lo que contiene la dimensión de ventas, la diferencia radica en que necesita ser un poco modificada.

In [ ]:
df_factTable=df_ventas

In [ ]:
df_ventas['year'] = pd.DatetimeIndex(df_ventas['fecha']).year
df_factTable['month'] = pd.DatetimeIndex(df_factTable['fecha']).month
df_factTable['day'] = pd.DatetimeIndex(df_factTable['fecha']).day
df_factTable['id_fecha'] = df_factTable['year'].astype(str) + df_factTable['month'].astype(str)
df_factTable['id_fecha'] = df_factTable['id_fecha'].astype(str) + df_factTable['day'].astype(str)
df_factTable = df_factTable.drop(['year', 'month', 'day', 'hour','minute', 'fecha'], axis=1)
df_factTable.head(3)

,id_ventas,id_movies,id_client,id_payment,total,id_fecha
0,1,357,148,1,15,202311
1,2,7082,265,3,40,202311
2,3,2512,187,2,5,202311


Ahora agregaré el indice del fact, de manera que sea autonumerica

In [ ]:
if 'id_fact' not in df_factTable.columns:
    # agregar una columna de ID autonumérico utilizando reset_index()
    df_factTable.reset_index(inplace=True)
    df_factTable.rename(columns={'index': 'id_fact'}, inplace=True)
    df_factTable['id_fact'] += 1

In [ ]:
df_factTable =df_factTable.drop(['id_ventas'], axis=1)
df_factTable.head(3)

,id_fact,id_movies,id_client,id_payment,total,id_fecha
0,1,357,148,1,15,202311
1,2,7082,265,3,40,202311
2,3,2512,187,2,5,202311


#Creación del Modelo Estrella con las Dimensiones y el Data-Warehouse

In [ ]:
config = configparser.ConfigParser()
config.read('escec_dataWarehouse.cfg')

['escec_dataWarehouse.cfg']

In [ ]:
aws_conn = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                    aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                    region_name='us-east-2')
print(aws_conn)

In [ ]:
rdsInstanceIds = []

response = aws_conn.describe_db_instances()
for resp in response['DBInstances']:
    rdsInstanceIds.append(resp['DBInstanceIdentifier'])
    db_instance_status = resp['DBInstanceStatus']

print(f"DBInstanceIds {rdsInstanceIds}")

DBInstanceIds ['dw-moviestore', 'mymoviestore']


Le colocamos el nombre a nuestra DW y esperamos unos minutos.

In [ ]:
rdsIdentifier = 'dw-moviestore'

In [ ]:
try:
    response = aws_conn.create_db_instance(
            AllocatedStorage=10,
            DBName=config.get('RDS_MYSQL', 'DB_NAME'),
            DBInstanceIdentifier=rdsIdentifier,
            DBInstanceClass="db.t3.micro",
            Engine="mysql",
            MasterUsername=config.get('RDS_MYSQL', 'DB_USER'),
            MasterUserPassword=config.get('RDS_MYSQL', 'DB_PASSWORD'),
            Port=int(config.get('RDS_MYSQL', 'DB_PORT')),
            VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')],
            PubliclyAccessible=True
        )
    print(response)
except aws_conn.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La Instancia de Base de Datos ya Existe.")

La Instancia de Base de Datos ya Existe.


Antes de conectarnos recordemos esperar el tiempo suficiente para agregar la nueva instancia y agregar el puerto 3306 al grupo de seguridad.

In [ ]:
try:
     instances = aws_conn.describe_db_instances(DBInstanceIdentifier=rdsIdentifier)
     RDS_DW_HOST = instances.get('DBInstances')[0].get('Endpoint').get('Address')
     print(RDS_DW_HOST)
except Exception as ex:
     print("La instancia de base de datos no existe o aun no se ha terminado de crear.")
     print(ex)

dw-moviestore.c6j34omip28z.us-east-2.rds.amazonaws.com


Creamos las tablas del DW.

In [ ]:
!pip install mysql-connector-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sql_dw_query
import mysql.connector as mysqlC
try:
    myDw = mysqlC.connect(
    host=RDS_DW_HOST, 
    user=config.get('RDS_MYSQL', 'DB_USER'),
    password=config.get('RDS_MYSQL', 'DB_PASSWORD'),
    database=config.get('RDS_MYSQL', 'DB_NAME')
    )

    mycursor = myDw.cursor()
    # Ejecutar las consultas
    try:
        mycursor.execute("CREATE TABLE IF NOT EXISTS custumer (id_client int PRIMARY KEY, name varchar(50) DEFAULT NULL, gender varchar(50)  DEFAULT NULL, custumer_type varchar(50) DEFAULT NULL);")
        mycursor.execute("CREATE TABLE IF NOT EXISTS movies (id_movies INT PRIMARY KEY, show_type TEXT NULL DEFAULT NULL, title TEXT NULL DEFAULT NULL, director TEXT NULL DEFAULT NULL, casting TEXT NULL DEFAULT NULL, country_name TEXT NULL DEFAULT NULL, date_added DATE NULL DEFAULT NULL, release_year BIGINT NULL DEFAULT NULL, rating TEXT NULL DEFAULT NULL, duration TEXT NULL DEFAULT NULL, listed_in TEXT NULL DEFAULT NULL, description TEXT NULL DEFAULT NULL, price INT NULL DEFAULT NULL);")
        mycursor.execute("CREATE TABLE IF NOT EXISTS payment_methods (id_payment INT PRIMARY KEY, name varchar(50));")
        mycursor.execute("CREATE TABLE IF NOT EXISTS fact_table (id_fact INT PRIMARY KEY, id_movies INT NOT NULL, id_client INT NOT NULL, id_payment INT NOT NULL, total INT, id_fecha DATE, CONSTRAINT fk_fact_dim_movies FOREIGN KEY (id_movies) REFERENCES movies(id_movies), CONSTRAINT fk_fact_dim_custumer FOREIGN KEY (id_client) REFERENCES custumer(id_client), CONSTRAINT fk_fact_dim_payment_methods FOREIGN KEY (id_payment) REFERENCES payment_methods(id_payment));")
    except mysqlC.connector.Error as err:
        print("Error al ejecutar la consulta: {}".format(err))
    

    #mycursor.execute(sql_dw_query.CREATE_DW, multi=True)   ->>>>>>>>Este comando me dió error, asi que obté por ejecutar tabla a tabla.

    myDw.commit()
    print("Data Warehouse Creado Exitosamente")
except Exception as ex:
    print("ERROR: Error al crear la base de datos.")
    print(ex)

Data Warehouse Creado Exitosamente


Poblamos las tablas del DW

In [ ]:
mysql_driver = f"""mysql+pymysql://{config.get('RDS_MYSQL', 'DB_USER')}:{config.get('RDS_MYSQL', 'DB_PASSWORD')}@{RDS_DW_HOST}:{config.get('RDS_MYSQL', 'DB_PORT')}/{config.get('RDS_MYSQL', 'DB_NAME')}"""  

##Insertamos Custumers

In [ ]:
!pip install pymysql   #Este package hacia falta.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 KB 2.5 MB/s eta 0:00:00


In [ ]:
dim_custumer.to_sql('custumer', mysql_driver, index=False, if_exists='append')

300

##Insertamos las Movies

In [ ]:
dim_movies.to_sql('movies',mysql_driver, index=False, if_exists='append')

8807

##Insertamos los payment_methods

In [ ]:
dim_payment_methods.to_sql('payment_methods',mysql_driver, index=False, if_exists='append')

3

##Insertamos la fact_table

In [ ]:
df_factTable.to_sql('fact_table',mysql_driver, index=False, if_exists='replace')

590